# Experiment and path specification

In [2]:
# Specify the experiment for analysis
experiment='xppl2816'
runNumber = 74

# Set the installation and output path
import os
os.environ['INSTALLPATH']='/reg/neh/home4/mrware/PhilTransA-TRXS-Limits'
os.environ['OUTPUTPATH']= '/reg/d/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [3]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *
# Determine current user
currentUser, error = unixCMD("echo $USER")
print(currentUser)
os.environ['RESULTSPATH']= ('/reg/d/psdm/%s/%s/res/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

mrware



OSError: [Errno 17] File exists: '/reg/d/psdm/xpp/xppl2816/res/mrware'

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [4]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsTools')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

/reg/d/psdm/xpp/xppl2816/scratch/Memories/getCSPADcoords remembered!


## Grabbing point data across nodes

In [ ]:

detDict = {'opaint':{'name':'XppEnds_Ipm0','get-function': 'getIPM' },
           'xint3':{'name':'XppSb3_Ipm','get-function': 'getIPM' },
           'xpos3':{'name':'XppSb3_Ipm','get-function': 'getXPos' },
           'ypos3':{'name':'XppSb3_Ipm','get-function': 'getYPos' },
           'xint2':{'name':'XppSb2_Ipm','get-function': 'getIPM' },
           'xpos2':{'name':'XppSb2_Ipm','get-function': 'getXPos' },
           'ypos2':{'name':'XppSb2_Ipm','get-function': 'getYPos' },
           'xrayEnergy':{'name':'SIOC:SYS0:ML00:AO541','get-function': 'getXrayEnergy' },
          'stageencoder':{'name':'XppEndstation.0:USDUSB.0','get-function':'getStageEncoder'},
          'ttfltpos':{'name':'XPP:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
          'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'fltposfwhm':{'name':'XPP:TIMETOOL:FLTPOSFWHM', 'get-function': 'getFltPosFWHM'},
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'},
          'cspadSum':{'name':'fiducials', 'get-function': 'getCSPADsum'},
          'cspadMedian':{'name':'fiducials', 'get-function': 'getCSPADmedian'}}


# Lookup={'EbeamCharge':None, 'XrayPulseEnergy':None, 'TTRefAmpl':None, 'TTFltPos':None,
#     'TTAmpl':None, 'TTFltPosFWHM':None, 'CSPAD':{'Integrated':None, 'Median':None}, 
#     'XrayIpm3':{'X':None, 'Y':None,'Sum':None},
#     'XrayIpm2':{'X':None, 'Y':None,'Sum':None}, 'OPAIpm':None}

batchThreads = batchPointDataGrabber( detDict, eventMax=1e5, experiment='xppl2816', runNumber=runNumber, rank=100 )
batchThreads.Queue = 'psanaq'
batchThreads.start()

## Check job status

In [ ]:
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata

## Kill threads if they die

In [ ]:
# batchThreads.requestStop()
# time.sleep(5)
# bkill(killAll=True)

In [ ]:
batchThreads.status

## Drop empty array values

In [ ]:
pointData = batchThreads.detArrays

def dropEmpty(pointData):

    filledBin = (pointData['seconds'] > 0)

    for key in pointData.keys():
        pointData[key]=pointData[key][filledBin] 
        
    return pointData

pointData = dropEmpty(pointData)


## Save/Load point data

In [5]:
# save_obj( pointData, os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )
pointData = load_obj( os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )

/reg/d/psdm/xpp/xppl2816/res/mrware/pointData-run-74 remembered!


## Plot histogram of result

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['ebeamcharge'], bins=100,normed=False, alpha=0.5);